## 1. 프로젝트 생성

In [1]:
# !pip install fake_useragent scrapy-fake-useragent

In [2]:
import scrapy, requests, re
import pandas as pd
from scrapy.http import TextResponse
from selenium import webdriver

/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
!rm -rf musinsa_2
!scrapy startproject musinsa_2

New Scrapy project 'musinsa_2', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/notebooks/project/crawling-repo-3/musinsa_2

You can start your first spider with:
    cd musinsa_2
    scrapy genspider example example.com


## 2. items.py

In [4]:
%%writefile musinsa_2/musinsa_2/items.py
import scrapy


class Musinsa2Item(scrapy.Item):
    item_id = scrapy.Field()
    most_age = scrapy.Field()
    most_sex = scrapy.Field()
    colors = scrapy.Field()
    sizes = scrapy.Field()

Overwriting musinsa_2/musinsa_2/items.py


## 3. spider.py
### selenium part

In [5]:
%%writefile musinsa_2/musinsa_2/spiders/spider.py

import scrapy
from musinsa_2.items import Musinsa2Item
from selenium import webdriver
from fake_useragent import UserAgent


class Spider(scrapy.Spider):
    name = "Musinsa2"
    allow_domain = ["musinsa.com"]
    start_urls = ["https://search.musinsa.com/category/001003"]
    custom_settings = {
        'CONCURRENT_REQUESTS': 3,
        'DOWNLOAD_DELAY': 6.0,
        'AUTOTHROTTLE_ENABLED': True,
        "DOWNLOADER_MIDDLEWARES": {
            "scrapy.downloadermiddlewares.useragent.UserAgentMiddleware": None,
            "scrapy_fake_useragent.middleware.RandomUserAgentMiddleware": 400,
        }
    }
    
    def parse(self, response):
        links = response.xpath('//*[@id="searchList"]/li/div[contains(@class,"li_inner")]\
/div[2]/p[2]/a/@href').extract()
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):     
        item = Musinsa2Item()
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        options.add_argument("user-agent={}".format(UserAgent().Chrome))
        driver = webdriver.Chrome(options=options)        
        driver.get(response.url)       
        
        try:
            item["most_age"] = driver.find_elements_by_xpath('//*[@id="graph_summary_area"]/strong[1]')[0].text
        except:
            item["most_age"] = 0
        try:
            item["most_sex"] = driver.find_elements_by_xpath('//*[@id="graph_summary_area"]/span[1]')[0].text
        except:
            item["most_sex"] = 0

        option1_ls = driver.find_elements_by_xpath('//*[@id="option1"]/option[not(@value="")]')
        option1 = [option.get_attribute("value") for option in option1_ls]
        driver.find_element_by_xpath('//*[@id="option1"]/option[not(@value="")]').click()
        option2_ls = driver.find_elements_by_xpath('//*[@id="option2"]/option[not(@value="")]')
        option2 = [option.get_attribute("value") for option in option2_ls]

        for_size = ['90', '95', 'S', 'M', 'L', 'LARGE', '1', 'M_쭈리']
        if any(x in for_size for x in option1):
            item["sizes"] = option1
            item["colors"] = option2
        else:
            item["sizes"] = option2
            item["colors"] = option1

        driver.quit()
        
        item["item_id"] = response.url.split('/')[-1]
        
        yield item

Writing musinsa_2/musinsa_2/spiders/spider.py


## 4. run scrapy
### selenium part

In [6]:
%%writefile run2.sh
cd musinsa_2
scrapy crawl Musinsa2 -o ./datas/top_2.csv

Writing run2.sh


### settings.py 변경

In [7]:
!head -n 20 musinsa_2/musinsa_2/settings.py | tail -n 2

# Obey robots.txt rules
ROBOTSTXT_OBEY = True


In [8]:
!sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' musinsa_2/musinsa_2/settings.py

In [9]:
!head -n 20 musinsa_2/musinsa_2/settings.py | tail -n 2

# Obey robots.txt rules
ROBOTSTXT_OBEY = False


In [10]:
%%time
!/bin/bash run2.sh

2021-03-01 16:16:55 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: musinsa_2)
2021-03-01 16:16:55 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Feb 27 2021, 12:08:28) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-01 16:16:55 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-01 16:16:55 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'BOT_NAME': 'musinsa_2',
 'CONCURRENT_REQUESTS': 3,
 'DOWNLOAD_DELAY': 6.0,
 'NEWSPIDER_MODULE': 'musinsa_2.spiders',
 'SPIDER_MODULES': ['musinsa_2.spiders']}
2021-03-01 16:16:55 [scrapy.extensions.telnet] INFO: Telnet Password: 0479df2ec592257c
2021-03-01 16:16:55 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.

2021-03-01 16:17:01 [urllib3.connectionpool] DEBUG: http://127.0.0.1:49699 "POST /session/ad20f998804602f46477b20d28c22856/execute/sync HTTP/1.1" 200 13
2021-03-01 16:17:01 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-01 16:17:01 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:49699/session/ad20f998804602f46477b20d28c22856/execute/sync {"script": "return (function(){return function(){var d=this;function f(a){return\"string\"==typeof a};function h(a,b){this.code=a;this.a=l[a]||m;this.message=b||\"\";a=this.a.replace(/((?:^|\\s+)[a-z])/g,function(a){return a.toUpperCase().replace(/^[\\s\\xa0]+/g,\"\")});b=a.length-5;if(0>b||a.indexOf(\"Error\",b)!=b)a+=\"Error\";this.name=a;a=Error(this.message);a.name=this.name;this.stack=a.stack||\"\"}\n(function(){var a=Error;function b(){}b.prototype=a.prototype;h.b=a.prototype;h.prototype=new b;h.prototype.constructor=h;h.a=function(b,c,g){for(var e=Array(arguments.length-2),k=2;k<arguments

2021-03-01 16:17:02 [urllib3.connectionpool] DEBUG: http://127.0.0.1:49699 "POST /session/ad20f998804602f46477b20d28c22856/execute/sync HTTP/1.1" 200 13
2021-03-01 16:17:02 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-01 16:17:02 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:49699/session/ad20f998804602f46477b20d28c22856/element {"using": "xpath", "value": "//*[@id=\"option1\"]/option[not(@value=\"\")]"}
2021-03-01 16:17:02 [urllib3.connectionpool] DEBUG: http://127.0.0.1:49699 "POST /session/ad20f998804602f46477b20d28c22856/element HTTP/1.1" 200 88
2021-03-01 16:17:02 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-01 16:17:02 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:49699/session/ad20f998804602f46477b20d28c22856/element/ec1f7214-1a62-4daa-9c1f-fc0737fec779/click {"id": "ec1f7214-1a62-4daa-9c1f-fc0737fec779"}
2021-03-01 16:17:03 [urllib3.connectionpool] DEBUG: 

2021-03-01 16:17:09 [urllib3.connectionpool] DEBUG: http://127.0.0.1:33537 "POST /session/94a97252f044d22474fcc009cbb569ff/element/2cfff9be-dc0e-4a4e-b776-4604760e3ded/click HTTP/1.1" 200 14
2021-03-01 16:17:09 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-01 16:17:09 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:33537/session/94a97252f044d22474fcc009cbb569ff/elements {"using": "xpath", "value": "//*[@id=\"option2\"]/option[not(@value=\"\")]"}
2021-03-01 16:17:09 [urllib3.connectionpool] DEBUG: http://127.0.0.1:33537 "POST /session/94a97252f044d22474fcc009cbb569ff/elements HTTP/1.1" 200 406
2021-03-01 16:17:09 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-01 16:17:09 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:33537/session/94a97252f044d22474fcc009cbb569ff/execute/sync {"script": "return (function(){return function(){var d=this;function f(a){return\"string\"==type

2021-03-01 16:17:10 [urllib3.connectionpool] DEBUG: http://127.0.0.1:33537 "POST /session/94a97252f044d22474fcc009cbb569ff/execute/sync HTTP/1.1" 200 15
2021-03-01 16:17:10 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-01 16:17:10 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:33537/session/94a97252f044d22474fcc009cbb569ff/execute/sync {"script": "return (function(){return function(){var d=this;function f(a){return\"string\"==typeof a};function h(a,b){this.code=a;this.a=l[a]||m;this.message=b||\"\";a=this.a.replace(/((?:^|\\s+)[a-z])/g,function(a){return a.toUpperCase().replace(/^[\\s\\xa0]+/g,\"\")});b=a.length-5;if(0>b||a.indexOf(\"Error\",b)!=b)a+=\"Error\";this.name=a;a=Error(this.message);a.name=this.name;this.stack=a.stack||\"\"}\n(function(){var a=Error;function b(){}b.prototype=a.prototype;h.b=a.prototype;h.prototype=new b;h.prototype.constructor=h;h.a=function(b,c,g){for(var e=Array(arguments.length-2),k=2;k<arguments

...

2021-03-01 16:28:44 [urllib3.connectionpool] DEBUG: http://127.0.0.1:45399 "POST /session/623e68d7acba5792eed3059bbaf8c061/execute/sync HTTP/1.1" 200 13
2021-03-01 16:28:44 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-01 16:28:44 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:45399/session/623e68d7acba5792eed3059bbaf8c061/execute/sync {"script": "return (function(){return function(){var d=this;function f(a){return\"string\"==typeof a};function h(a,b){this.code=a;this.a=l[a]||m;this.message=b||\"\";a=this.a.replace(/((?:^|\\s+)[a-z])/g,function(a){return a.toUpperCase().replace(/^[\\s\\xa0]+/g,\"\")});b=a.length-5;if(0>b||a.indexOf(\"Error\",b)!=b)a+=\"Error\";this.name=a;a=Error(this.message);a.name=this.name;this.stack=a.stack||\"\"}\n(function(){var a=Error;function b(){}b.prototype=a.prototype;h.b=a.prototype;h.prototype=new b;h.prototype.constructor=h;h.a=function(b,c,g){for(var e=Array(arguments.length-2),k=2;k<arguments

2021-03-01 16:28:44 [urllib3.connectionpool] DEBUG: http://127.0.0.1:45399 "POST /session/623e68d7acba5792eed3059bbaf8c061/element HTTP/1.1" 200 88
2021-03-01 16:28:44 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-01 16:28:44 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:45399/session/623e68d7acba5792eed3059bbaf8c061/element/80ff202b-2ae8-4399-a7b3-acc9f33923c7/click {"id": "80ff202b-2ae8-4399-a7b3-acc9f33923c7"}
2021-03-01 16:28:44 [urllib3.connectionpool] DEBUG: http://127.0.0.1:45399 "POST /session/623e68d7acba5792eed3059bbaf8c061/element/80ff202b-2ae8-4399-a7b3-acc9f33923c7/click HTTP/1.1" 200 14
2021-03-01 16:28:44 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-01 16:28:44 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:45399/session/623e68d7acba5792eed3059bbaf8c061/elements {"using": "xpath", "value": "//*[@id=\"option2\"]/option[not(@value=\"\")]"}
2021-03-01 16

2021-03-01 16:28:51 [urllib3.connectionpool] DEBUG: http://127.0.0.1:56231 "POST /session/65cf93e929ce2ccf2312cabdd6c8f84c/execute/sync HTTP/1.1" 200 13
2021-03-01 16:28:51 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-01 16:28:51 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:56231/session/65cf93e929ce2ccf2312cabdd6c8f84c/execute/sync {"script": "return (function(){return function(){var d=this;function f(a){return\"string\"==typeof a};function h(a,b){this.code=a;this.a=l[a]||m;this.message=b||\"\";a=this.a.replace(/((?:^|\\s+)[a-z])/g,function(a){return a.toUpperCase().replace(/^[\\s\\xa0]+/g,\"\")});b=a.length-5;if(0>b||a.indexOf(\"Error\",b)!=b)a+=\"Error\";this.name=a;a=Error(this.message);a.name=this.name;this.stack=a.stack||\"\"}\n(function(){var a=Error;function b(){}b.prototype=a.prototype;h.b=a.prototype;h.prototype=new b;h.prototype.constructor=h;h.a=function(b,c,g){for(var e=Array(arguments.length-2),k=2;k<arguments

2021-03-01 16:28:52 [urllib3.connectionpool] DEBUG: http://127.0.0.1:56231 "POST /session/65cf93e929ce2ccf2312cabdd6c8f84c/execute/sync HTTP/1.1" 200 13
2021-03-01 16:28:52 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-01 16:28:52 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:56231/session/65cf93e929ce2ccf2312cabdd6c8f84c/execute/sync {"script": "return (function(){return function(){var d=this;function f(a){return\"string\"==typeof a};function h(a,b){this.code=a;this.a=l[a]||m;this.message=b||\"\";a=this.a.replace(/((?:^|\\s+)[a-z])/g,function(a){return a.toUpperCase().replace(/^[\\s\\xa0]+/g,\"\")});b=a.length-5;if(0>b||a.indexOf(\"Error\",b)!=b)a+=\"Error\";this.name=a;a=Error(this.message);a.name=this.name;this.stack=a.stack||\"\"}\n(function(){var a=Error;function b(){}b.prototype=a.prototype;h.b=a.prototype;h.prototype=new b;h.prototype.constructor=h;h.a=function(b,c,g){for(var e=Array(arguments.length-2),k=2;k<arguments

2021-03-01 16:28:52 [urllib3.connectionpool] DEBUG: http://127.0.0.1:56231 "POST /session/65cf93e929ce2ccf2312cabdd6c8f84c/execute/sync HTTP/1.1" 200 15
2021-03-01 16:28:52 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-01 16:28:52 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:56231/session/65cf93e929ce2ccf2312cabdd6c8f84c/element {"using": "xpath", "value": "//*[@id=\"option1\"]/option[not(@value=\"\")]"}
2021-03-01 16:28:52 [urllib3.connectionpool] DEBUG: http://127.0.0.1:56231 "POST /session/65cf93e929ce2ccf2312cabdd6c8f84c/element HTTP/1.1" 200 88
2021-03-01 16:28:52 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-01 16:28:52 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:56231/session/65cf93e929ce2ccf2312cabdd6c8f84c/element/d61d6689-742c-4741-8164-16a249e0e89b/click {"id": "d61d6689-742c-4741-8164-16a249e0e89b"}
2021-03-01 16:28:52 [urllib3.connectionpool] DEBUG: 

In [11]:
sele = pd.read_csv('./musinsa_2/datas/top_2.csv')

In [12]:
len(sele)

85